#Setup

In [ ]:
!pip install pkbar

In [ ]:
from google.colab import drive
from google.colab import files
import sys
import time

drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/MS Thesis/PSGD Paper/'
results_dir = base_dir + 'CNN/results/'
logs_dir = base_dir + 'log'
sys.path.append(base_dir)
import preconditioned_stochastic_gradient_descent as psgd 


Mounted at /content/gdrive/


In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.autograd import grad
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import time
import tqdm
import pkbar
import math

from tabulate import tabulate
import scipy.io
from sklearn import metrics
import plotly.express as px
from torchsummary import summary


# Functions

In [ ]:
def plot_loss_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis_type="log")
  fig.show()
  fig.write_html(results_dir + title + ".html")

def plot_acc_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis=dict(range=[0.97, 1]))
  fig.show()
  fig.write_html(results_dir + title + ".html")



In [ ]:
np.random.seed(0)

# Parameter Settings
BATCH_SIZE = 64
test_BATCH_SIZE = 1000
EPOCHS = 20
GAP = 100

# Data Download

In [ ]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,           
                       transform=transforms.Compose([                       
                               transforms.ToTensor()])),    
                        batch_size=BATCH_SIZE, shuffle=True, num_workers = 4, pin_memory = True)
test_loader = torch.utils.data.DataLoader(    
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()])),    
                        batch_size=test_BATCH_SIZE, shuffle=True, num_workers=4, pin_memory = True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning:

The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)



In [ ]:
n_batches = np.ceil(len(train_loader.dataset)/BATCH_SIZE)
n_test_batches = np.ceil(len(test_loader.dataset)/test_BATCH_SIZE)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
torch.cuda.device_count()
torch.cuda.get_device_name(0)

Running on the GPU




'Tesla T4'

# Model

In [ ]:
"""input image size for the original LeNet5 is 32x32, here is 28x28"""
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
def initialize_weights():
    W1 = torch.nn.init.xavier_uniform_((0.1*torch.randn(1*5*5+1,  6)).clone().detach().requires_grad_(True)).to(device)
    W2 = torch.nn.init.xavier_uniform_((0.1*torch.randn(6*5*5+1,  16)).clone().detach().requires_grad_(True)).to(device)
    W3 = torch.nn.init.xavier_uniform_((0.1*torch.randn(16*4*4+1, 120)).clone().detach().requires_grad_(True)).to(device)
    W4 = torch.nn.init.xavier_uniform_((0.1*torch.randn(120+1,    84)).clone().detach().requires_grad_(True)).to(device)
    W5 = torch.nn.init.xavier_uniform_((0.1*torch.randn(84+1,     10)).clone().detach().requires_grad_(True)).to(device)
    Ws = [W1, W2, W3, W4, W5]
    return Ws

def LeNet5(x, return_all = False): 
    W1, W2, W3, W4, W5 = Ws
    x1 = F.relu(F.conv2d(x, W1[:-1].view(6,1,5,5), bias=W1[-1]))
    x2 = F.max_pool2d(x1, 2)
    x3 = F.relu(F.conv2d(x2, W2[:-1].view(16,6,5,5), bias=W2[-1]))
    x4 = F.max_pool2d(x3, 2)
    x5 = F.relu(x4.view(-1, 16*4*4).mm(W3[:-1]) + W3[-1])
    x6 = F.relu(x5.mm(W4[:-1]) + W4[-1])
    # x = F.dropout(x, 0.2, training = True)
    y = x6.mm(W5[:-1]) + W5[-1]
    if return_all:
      return F.log_softmax(y, dim = 1), [x, x1, x2, x3, x4, x5, x6, y]
    return F.log_softmax(y, dim=1)


# Loss Function

In [ ]:
def train_loss(data, target):
    y = LeNet5(data)
    loss = F.nll_loss(y, target)
    # loss = F.cross_entropy(y, target)
    _, max_indices = torch.max(y, dim = 1)
    accuracy = (max_indices == target).sum(dtype=torch.float32)/max_indices.size(0)
    return loss, accuracy

def test_loss():
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            y = LeNet5(data)
            loss += F.nll_loss(y, target)
            _, pred = torch.max(y, dim=1)
            accuracy += (pred == target).sum(dtype=torch.float32)/pred.size(0)
    return loss.item()/n_test_batches, accuracy.item()/n_test_batches

def update_lambda(loss1, loss2, M, lambd, omega):
    
    r = abs(loss2 - loss1)/(M)
    # print(r, M, lambd)
    if r > 3/4:
      lambd = lambd*omega
    elif r < 1/4:
      lambd = lambd / omega
    return lambd
    
def save_start_condition(trainlosslist, testlosslist,trainacclist, testacclist, timelist):
    trainloss = 0.0
    trainacc = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      loss, accuracy = train_loss(data, target)
      trainloss += loss
      trainacc += accuracy
      
  
    timelist.append(0)

    testloss, testacc = test_loss()

    trainlosslist.append(trainloss.item()/n_batches)
    trainacclist.append(trainacc.item()/n_batches)
    testlosslist.append(testloss)
    testacclist.append(testacc)
    print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    .format(0, trainlosslist[-1], testlosslist[-1], trainacclist[-1], testacclist[-1],np.sum(timelist)))

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()

for batch_idx, (data, target) in enumerate(train_loader):
  data, target = data.to(device), target.to(device)
  loss, _ = train_loss(data, target)
  grads = grad(loss, Ws)
  print(grads)
  s = [torch.mean(g) for g in grads]
  print(s)
# print(x.shape)
# grads = grad()

# ALL TEST

## SGD

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws)
        
        trainloss += loss
        trainacc += accuracy
        
        with torch.no_grad():
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*grads[i]

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)

    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])

    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'sgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

Epoch: 0; train loss: 2.4869145310001333; test loss: 2.490052032470703, train_accuracy: 0.09931369936034115, test_accuracy:0.10080000162124633, time: 0
Epoch: 1/20
938/938 [==============================] - 9s 9ms/step - loss: 0.2063 - acc: 0.9332 - val_loss: 0.0707 - val_acc: 0.9761
Epoch: 2/20
938/938 [==============================] - 9s 9ms/step - loss: 0.0589 - acc: 0.9814 - val_loss: 0.0612 - val_acc: 0.9791
Epoch: 3/20
938/938 [==============================] - 9s 10ms/step - loss: 0.0403 - acc: 0.9878 - val_loss: 0.0405 - val_acc: 0.9868
Epoch: 4/20
938/938 [==============================] - 9s 9ms/step - loss: 0.0319 - acc: 0.9905 - val_loss: 0.0380 - val_acc: 0.9870
Epoch: 5/20
938/938 [==============================] - 9s 9ms/step - loss: 0.0276 - acc: 0.9916 - val_loss: 0.0339 - val_acc: 0.9883
Epoch: 6/20
938/938 [==============================] - 9s 9ms/step - loss: 0.0249 - acc: 0.9930 - val_loss: 0.0330 - val_acc: 0.9883
Epoch: 7/20
938/938 [============================

## Adam

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()

m0 = [torch.zeros(W.shape).to(device) for W in Ws]
v0 = [torch.zeros(W.shape).to(device) for W in Ws]
step_size = 0.005
cnt = 0
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws)#, create_graph=True)
        trainloss += loss
        trainacc += accuracy
        

        with torch.no_grad():
            lmbd = min(cnt/(cnt+1), 0.9)
            m0 = [lmbd*old + (1.0-lmbd)*new for (old, new) in zip(m0, grads)]
            lmbd = min(cnt/(cnt+1), 0.999)
            v0 = [lmbd*old + (1.0-lmbd)*new*new for (old, new) in zip(v0, grads)]
            for i in range(len(Ws)):
                Ws[i] -= step_size*(m0[i]/torch.sqrt(v0[i] + 1e-8))
            cnt = cnt + 1
        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1    
        
    t1 = time.time() - t0
    times.append(t1)

    testloss, testacc = test_loss()

    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    # .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

Epoch: 0; train loss: 2.4869145310001333; test loss: 2.490052032470703, train_accuracy: 0.09931369936034115, test_accuracy:0.10080000162124633, time: 0
Epoch: 1/20
938/938 [==============================] - 9s 10ms/step - loss: 0.1467 - acc: 0.9554 - val_loss: 0.0799 - val_acc: 0.9754
Epoch: 2/20
938/938 [==============================] - 9s 10ms/step - loss: 0.0478 - acc: 0.9853 - val_loss: 0.0407 - val_acc: 0.9871
Epoch: 3/20
938/938 [==============================] - 9s 10ms/step - loss: 0.0274 - acc: 0.9915 - val_loss: 0.0402 - val_acc: 0.9873
Epoch: 4/20
938/938 [==============================] - 9s 9ms/step - loss: 0.0168 - acc: 0.9947 - val_loss: 0.0355 - val_acc: 0.9902
Epoch: 5/20
938/938 [==============================] - 9s 10ms/step - loss: 0.0110 - acc: 0.9969 - val_loss: 0.0306 - val_acc: 0.9906
Epoch: 6/20
938/938 [==============================] - 9s 10ms/step - loss: 0.0070 - acc: 0.9984 - val_loss: 0.0333 - val_acc: 0.9907
Epoch: 7/20
938/938 [========================

## Full Kronecker

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.05
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'Kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

Epoch: 0; train loss: 2.4869145310001333; test loss: 2.490052032470703, train_accuracy: 0.09931369936034115, test_accuracy:0.10080000162124633, time: 0
Epoch: 1/20
938/938 [==============================] - 16s 17ms/step - loss: 0.1667 - acc: 0.9498 - val_loss: 0.0518 - val_acc: 0.9827
Epoch: 2/20
938/938 [==============================] - 16s 17ms/step - loss: 0.0391 - acc: 0.9880 - val_loss: 0.0453 - val_acc: 0.9864
Epoch: 3/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0209 - acc: 0.9935 - val_loss: 0.0385 - val_acc: 0.9877
Epoch: 4/20
938/938 [==============================] - 16s 17ms/step - loss: 0.0109 - acc: 0.9968 - val_loss: 0.0321 - val_acc: 0.9906
Epoch: 5/20
938/938 [==============================] - 16s 17ms/step - loss: 0.0062 - acc: 0.9987 - val_loss: 0.0308 - val_acc: 0.9911
Epoch: 6/20
938/938 [==============================] - 16s 17ms/step - loss: 0.0037 - acc: 0.9994 - val_loss: 0.0313 - val_acc: 0.9917
Epoch: 7/20
938/938 [=================

## Damped Kronecker

In [ ]:
_tiny = 1.2e-38 
 # pi = (torch.trace(Ql)*Qr.shape[0])/(torch.trace(Qr)*Ql.shape[0])
    # 

    
def precond_grad_kron(Ql, Qr, Grad):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd**0.5))*IR)

    return P1.mm(Grad).mm(P2)


In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.05
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

lambd = 1
update_after = 1
omega = (19/20)**update_after

eta = 1e-5
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.nll_loss(LeNet5(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)

    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'Kron_damped.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

Epoch: 0; train loss: 2.4869145310001333; test loss: 2.490052032470703, train_accuracy: 0.09931369936034115, test_accuracy:0.10080000162124633, time: 0
Epoch: 1/20
938/938 [==============================] - 18s 19ms/step - loss: 0.1401 - acc: 0.9553 - val_loss: 0.0453 - val_acc: 0.9838
Epoch: 2/20
938/938 [==============================] - 18s 19ms/step - loss: 0.0342 - acc: 0.9895 - val_loss: 0.0462 - val_acc: 0.9848
Epoch: 3/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0170 - acc: 0.9945 - val_loss: 0.0318 - val_acc: 0.9900
Epoch: 4/20
938/938 [==============================] - 18s 19ms/step - loss: 0.0068 - acc: 0.9981 - val_loss: 0.0319 - val_acc: 0.9908
Epoch: 5/20
938/938 [==============================] - 18s 19ms/step - loss: 0.0029 - acc: 0.9994 - val_loss: 0.0311 - val_acc: 0.9916
Epoch: 6/20
938/938 [==============================] - 17s 19ms/step - loss: 0.0012 - acc: 0.9997 - val_loss: 0.0338 - val_acc: 0.9915
Epoch: 7/20
938/938 [=================

## MIXED

In [ ]:
def precond_kron(Ql, Qr, Pl, Pr, beta1, beta2):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd**0.5))*IR)

    Pl = beta1*Pl + beta2*P1 
    Pr = beta1*Pr + beta2*P2 

    return [P1, P2, Pl, Pr]

def precond_kron2(Ql, Qr, Pl, Pr, beta1, beta2):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    Pl = beta1*Pl + beta2*P1 
    Pr = beta1*Pr + beta2*P2 
    return [P1, P2, Pl, Pr]

def precond_grad_kron2(Pl, Pr, Grad):
    return Pl.mm(Grad).mm(Pr)

torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
Ps = [[0.1*torch.eye(W.shape[0]).to(device), 0.1*torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.05
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

lambd = 1
update_after = 1
omega = (19/20)**update_after
eta = 1e-5

beta1 = 0.9
beta2 = 0.1

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    n = 0
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
   
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            Ps = [precond_kron(q[0], q[1], p[0], p[1], beta1, beta2) for (q, p) in zip(Qs, Ps)]
            pre_grads = [precond_grad_kron2(p[2], p[3], g) for (p, g) in zip(Ps, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % 1 == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.nll_loss(LeNet5(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M, lambd, omega)
        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)

    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'mod_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

Epoch: 0; train loss: 2.4869145310001333; test loss: 2.490052032470703, train_accuracy: 0.09931369936034115, test_accuracy:0.10080000162124633, time: 0
Epoch: 1/20
938/938 [==============================] - 20s 21ms/step - loss: 0.1403 - acc: 0.9556 - val_loss: 0.0493 - val_acc: 0.9839
Epoch: 2/20
938/938 [==============================] - 19s 21ms/step - loss: 0.0354 - acc: 0.9894 - val_loss: 0.0415 - val_acc: 0.9874
Epoch: 3/20
938/938 [==============================] - 19s 21ms/step - loss: 0.0173 - acc: 0.9946 - val_loss: 0.0325 - val_acc: 0.9920
Epoch: 4/20
938/938 [==============================] - 20s 21ms/step - loss: 0.0068 - acc: 0.9980 - val_loss: 0.0320 - val_acc: 0.9917
Epoch: 5/20
938/938 [==============================] - 20s 21ms/step - loss: 0.0028 - acc: 0.9992 - val_loss: 0.0301 - val_acc: 0.9930
Epoch: 6/20
938/938 [==============================] - 20s 21ms/step - loss: 0.0013 - acc: 0.9996 - val_loss: 0.0334 - val_acc: 0.9935
Epoch: 7/20
938/938 [=================

##KFAC

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
from kfac import KFAC
import torch.nn as nn
import torch.optim as optim

class LeNet5_K(nn.Module):
    def __init__(self):
        super(LeNet5_K, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

def test_loss_K(model):
    model.eval()
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            y = model(data)
            loss += F.nll_loss(y, target)
            _, pred = torch.max(y, dim=1)
            accuracy += (pred == target).sum(dtype=torch.float32)/pred.size(0)
    return loss.item()/n_test_batches, accuracy/n_test_batches

model = LeNet5_K().to(device)
preconditioner = KFAC(model, 0.001, alpha=0.05)
lr0 = 0.01
optimizer = optim.SGD(model.parameters(), lr=lr0)
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    model.train()
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        _, max_ind = torch.max(output, dim = 1)
        accuracy = (max_ind == target).sum(dtype=torch.float32)/max_ind.size(0) 

        trainloss += loss
        trainacc += accuracy

        loss.backward()
        preconditioner.step()
        optimizer.step()

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1
        
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    
    lr0 = 0.01**(1/9)*lr0
    optimizer.param_groups[0]['lr'] = lr0
    testloss, testacc = test_loss_K(model)

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'KFAC.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

Epoch: 0; train loss: 2.4869777785181237; test loss: 2.490052032470703, train_accuracy: 0.09929704157782517, test_accuracy:0.1008000135421753, time: 0
Epoch: 1/20
938/938 [==============================] - 15s 16ms/step - loss: 0.1095 - acc: 0.9671 - val_loss: 0.0475 - val_acc: 0.9857
Epoch: 2/20
938/938 [==============================] - 15s 16ms/step - loss: 0.0256 - acc: 0.9922 - val_loss: 0.0352 - val_acc: 0.9900
Epoch: 3/20
938/938 [==============================] - 15s 16ms/step - loss: 0.0092 - acc: 0.9977 - val_loss: 0.0288 - val_acc: 0.9928
Epoch: 4/20
938/938 [==============================] - 15s 16ms/step - loss: 0.0033 - acc: 0.9995 - val_loss: 0.0291 - val_acc: 0.9922
Epoch: 5/20
938/938 [==============================] - 15s 16ms/step - loss: 0.0020 - acc: 0.9998 - val_loss: 0.0287 - val_acc: 0.9924
Epoch: 6/20
938/938 [==============================] - 15s 16ms/step - loss: 0.0016 - acc: 0.9999 - val_loss: 0.0288 - val_acc: 0.9925
Epoch: 7/20
938/938 [==================

## Shampoo

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[0.01*torch.eye(W.shape[0]).to(device), 0.01*torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.5
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, TrainAcc, TestAcc, times)

def matrix_power(matrix, power):
    # use CPU for svd for speed up
    matrix = matrix.cpu()
    u, s, v = torch.svd(matrix)
    return (u @ s.pow_(power).diag() @ v.t()).cuda()

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        
      
        data, target = data.to(device), target.to(device)
        loss, accuracy = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        trainacc += accuracy
        
        with torch.no_grad():
            Qs = [[q[0] + g.mm(g.t()), q[1] + (g.t()).mm(g)] for (q, g) in zip(Qs, grads)]
            inv_Qs = [[matrix_power(q[0], -1/4), matrix_power(q[1], -1/4)]for q in Qs]
            pre_grads = [q[0].mm(g).mm(q[1]) for (q, g) in zip(inv_Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

        kbar.update(n, values=[("loss", loss.item()), ("acc", accuracy.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    TrainAcc.append(trainacc.item()/n_batches)
    
    testloss, testacc = test_loss()

    TestLoss.append(testloss)
    TestAcc.append(testacc)
    kbar.add(1, values=[("val_loss", testloss), ("val_acc", testacc)])
    step_size = 0.1**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, train_accuracy: {}, test_accuracy:{}, time: {}'\
    #  .format(epoch, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'shampoo.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAcc,'TestAccuracy': TestAcc, 'Time':times})

Epoch: 0; train loss: 2.4869145310001333; test loss: 2.490052032470703, train_accuracy: 0.09931369936034115, test_accuracy:0.10080000162124633, time: 0
Epoch: 1/20
938/938 [==============================] - 24s 25ms/step - loss: 0.1137 - acc: 0.9646 - val_loss: 0.0472 - val_acc: 0.9847
Epoch: 2/20
938/938 [==============================] - 24s 25ms/step - loss: 0.0294 - acc: 0.9912 - val_loss: 0.0356 - val_acc: 0.9885
Epoch: 3/20
938/938 [==============================] - 24s 25ms/step - loss: 0.0156 - acc: 0.9956 - val_loss: 0.0367 - val_acc: 0.9882
Epoch: 4/20
938/938 [==============================] - 23s 25ms/step - loss: 0.0093 - acc: 0.9977 - val_loss: 0.0360 - val_acc: 0.9881
Epoch: 5/20
938/938 [==============================] - 23s 25ms/step - loss: 0.0062 - acc: 0.9986 - val_loss: 0.0384 - val_acc: 0.9884
Epoch: 6/20
938/938 [==============================] - 24s 25ms/step - loss: 0.0046 - acc: 0.9991 - val_loss: 0.0371 - val_acc: 0.9891
Epoch: 7/20
938/938 [=================

# Comparison

In [ ]:
opts = ['sgd','adam','KFAC', 'shampoo','Kron','Kron_damped', 'mod_psgd', 'mod_psgd1']

total_train_time = {}
opts_data = {}
times = {}
train_times = {}
test_times = {}
train_losses = {}
test_losses = {}
train_accs = {}
test_accs = {}


for opt in opts:
	opts_data[opt] = scipy.io.loadmat(results_dir+opt+'.mat')	

In [ ]:
colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#000000','#33E0FF', '#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']
# colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF','#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']

In [ ]:
for opt in opts:
  # print(opt)
  data = opts_data[opt]
  times[opt] = data.get('Time')
  train_times[opt] = np.cumsum(times[opt])
  test_times[opt] = np.cumsum(times[opt])
  total_train_time[opt] = np.sum(times[opt])
  train_losses[opt] = data.get('TrainLoss').reshape(EPOCHS+1,)
  train_accs[opt] = data.get('TrainAccuracy').reshape(EPOCHS+1,)
  test_losses[opt] = data.get('TestLoss').reshape(EPOCHS+1,)
  test_accs[opt] = data.get('TestAccuracy').reshape(EPOCHS+1)


In [ ]:
# plot train_losses vs Iterations
plot_loss_metrics(None,train_losses,'Train Loss vs EPOCHS', 'EPOCHS','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(None,test_losses,'Test Loss vs EPOCHS', 'EPOCHS','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_losses,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')

In [ ]:
# plot train_losses vs Iterations
plot_acc_metrics(None,train_accs,'Train Accuracy vs EPOCHS', 'EPOCHS','Train Accuracy')
# plot test_losses vs Iterations
plot_acc_metrics(None,test_accs,'Test Accuracy vs EPOCHS', 'EPOCHS','Test Accuracys')
# # plot test_losses vs Iterations
plot_acc_metrics(train_times,train_accs,'Train Accuracy vs Time', 'Time','Train Accuracy')
# plot test_losses vs Iterations
plot_acc_metrics(test_times,test_accs,'Test Accuracy vs Time', 'Time','Test Accuracy')